In [1]:
from read_json import load_from_json

In [2]:
objects, connections = load_from_json('./JSON/mental_imagery_extended.json')

ComponentMultiply does not have 2 inputs, but 0!
Source is a CM object: Component Multiply 29
[('name', 'Component Multiply 29'), ('comments', ''), ('inp size1', ['50', '50']), ('inp size2', ['50', '50'])]
Input size 1 and 2: ['50', '50'] ['50', '50']
Used as output size: ['50', '50'] 

Source is a CM object: Component Multiply 30
[('name', 'Component Multiply 30'), ('comments', ''), ('inp size1', ['50', '50']), ('inp size2', ['50', '50'])]
Input size 1 and 2: ['50', '50'] ['50', '50']
Used as output size: ['50', '50'] 

Source is a CM object: Component Multiply 33
[('name', 'Component Multiply 33'), ('comments', ''), ('inp size1', ['50', '50']), ('inp size2', ['50', '50'])]
Input size 1 and 2: ['50', '50'] ['50', '50']
Used as output size: ['50', '50'] 

Source is a CM object: Component Multiply 35
[('name', 'Component Multiply 35'), ('comments', ''), ('inp size1', ['50', '50']), ('inp size2', ['50', '50'])]
Input size 1 and 2: ['50', '50'] ['50', '50']
Used as output size: ['50', '50

In [3]:
# check if I have all objects now by checking if the numbers are the same as
# in the json file
object_counts = {}

for key in objects:
    if objects[key][0] not in object_counts.keys():
        object_counts[objects[key][0]] = 1
    else:
        object_counts[objects[key][0]] += 1
        
for key in object_counts:
    print('Number of %s: %i' %(key, object_counts[key]))

# --> Numbers check out

Number of cedar.dynamics.NeuralField: 90
Number of cedar.processing.sources.Boost: 23
Number of cedar.processing.ComponentMultiply: 52
Number of cedar.processing.sources.ConstMatrix: 8
Number of cedar.processing.steps.Convolution: 4
Number of cedar.processing.Flip: 4
Number of cedar.processing.sources.GaussInput: 16
Number of cedar.processing.Projection: 38
Number of cedar.processing.sources.SpatialTemplate: 24
Number of cedar.processing.StaticGain: 166


# Test simpler JSON file
To be sure that everything works correctly I added a small test architecture to the JSON files. It includes all scenarios of different group input and output connections, but with only a few objects to stay on top of things. 

In [4]:
test_objects, test_connections = load_from_json('./JSON/test_architecture.json')

In [5]:
for key in test_objects:
    print(test_objects[key][1][0][1])

Flip
Linear Spatial Template
Projection
Projection 2
Spatial Template
Group 1.Component Multiply
Group 1.Neural Field
Group 1.Static Gain
Group 2.Gauss Input
Group 2.Neural Field 2


In [6]:
test_connections

[[('source', 'Group 1.Static Gain.output'), ('target', 'Flip.input')],
 [('source', 'Group 2.Neural Field 2.sigmoided activation'),
  ('target', 'Projection.input')],
 [('source', 'Group 2.Neural Field 2.sigmoided activation'),
  ('target', 'Projection 2.input')],
 [('source', 'Linear Spatial Template.spatial pattern'),
  ('target', 'Group 1.Component Multiply.operands')],
 [('source', 'Spatial Template.spatial pattern'),
  ('target', 'Group 1.Component Multiply.operands')],
 [('source', 'Spatial Template.spatial pattern'),
  ('target', 'Group 1.Static Gain.input')],
 [('source', 'Group 1.Neural Field.sigmoided activation'),
  ('target', 'Group 2.Neural Field 2.input')],
 [('source', 'Group 1.Component Multiply.product'),
  ('target', 'Group 1.Neural Field.input')],
 [('source', 'Group 1.Static Gain.output'),
  ('target', 'Group 1.Neural Field.input')],
 [('source', 'Group 2.Gauss Input.Gauss input'),
  ('target', 'Group 2.Neural Field 2.input')]]

# Object Parameters

In [7]:
example_objects = {}
for key in objects:
    if objects[key][0] not in example_objects.keys():
        example_objects[objects[key][0]] = objects[key]
    

In [8]:
example_objects['cedar.processing.steps.Convolution']

('cedar.processing.steps.Convolution',
 [('name', 'Convolution'),
  ('kernels', ''),
  ('convolution',
   [('borderType', 'Zero'),
    ('mode', 'Same'),
    ('engine', [('type', 'cedar.aux.conv.OpenCV')]),
    ('alternate even kernel center', 'false')]),
  ('comments', ''),
  ('sizes', ['50', '50'])])

In [9]:
# objects['Target']

In [10]:
import parse_cedar_objects as parser

In [11]:
parser.const_matrix_parser(example_objects['cedar.processing.sources.ConstMatrix'])
print()
parser.flip_parser(example_objects['cedar.processing.Flip'])
print()
parser.gauss_input_parser(example_objects['cedar.processing.sources.GaussInput'])
print()
parser.projection_parser(example_objects['cedar.processing.Projection'])
print()
parser.spatial_template_parser(example_objects['cedar.processing.sources.SpatialTemplate'])
print()
parser.static_gain_parser(example_objects['cedar.processing.StaticGain'])
print()
parser.component_multiply_parser(example_objects['cedar.processing.ComponentMultiply'])
print()
parser.convolution_parser(example_objects['cedar.processing.steps.Convolution'])

ConstantMatrix with params: 
 name: Const Matrix 
 sizes: [50, 50] 
 value: 42

Flip with params: 
 name: Flip 
 sizes: [50, 50] 
 flip dimensions:  [True, True]

GaussInput with params: 
 name: Gauss Input 10 
 sizes: [50] 
 centers:  [25.0] 
 sigmas: [1.5] 
 amplitude: 8.8

Projection with params: 
 name: Projection 
 sizes_in: [50, 50, 50] 
 sizes_out: [50, 50] 
 compression_type: max 
 dimension_mapping: {'0': 0, '1': 1, '2': False}

SpatialTemplate with params: 
 name: Spatial Template 
 sizes: [50, 50] 
 invert_sides: True 
 horizontal_pattern: True 
 sigma_th_hor: 0.25 
 mu_r: 1.1 
 sigma_r: 3.0 
 sigma_sigmoid_fw: 0.0

StaticGain with params: 
 name: Static Gain 
 sizes: [50, 50] 
 gain_factor: 5.0

ComponentMultiply with params: 
 name: Component Multiply 
 inp_size1: [] 
 inp_size2: [50, 50]

Convolution with params: 
 name: Convolution 
 sizes: [50, 50] 
 border_type: zero-filled borders


In [12]:
parser.neural_field_parser(example_objects['cedar.dynamics.NeuralField'])

('sigmoid', [('type', 'cedar.aux.math.AbsSigmoid'), ('threshold', '0'), ('beta', '100')])
Neural Field with params: 
 name: Above Memory 
 sizes: [] 
 h: -5.0 
 tau: 100.0 
 c_glob: -0.01 
 border_type: zero-filled borders 
 input_noise_gain: 0.1 
 kernel: <cedar_modules.BoxKernel object at 0x7fc6119d6978> 
 nonlinearity: <cedar_modules.AbsSigmoid object at 0x7fc6119d6b00>
